### Code to find Table coordinates based on yolo

In [10]:
import cv2
import numpy as np
from ultralytics import YOLO
import torch

In [11]:
def yolo_on_video(model, video, start_frame, end_frame):
    cap = cv2.VideoCapture(video)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    frame_num = start_frame
    all_results = []
    while True:
        ret, frame = cap.read()
        if not ret or frame_num > end_frame:
            break

        # Run YOLO inference
        if torch.cuda.is_available():
            results = model(frame, stream=True, device="cuda")
        else:
            results = model(frame, stream=True, device="cpu")

        for r in results:
            all_results.append(r)
            annotated_frame = r.plot()
            cv2.imshow("YOLO Pose - Full", annotated_frame)

            keypoints = r.keypoints.cpu().numpy()  # (num_instances, num_keypoints, 3)
            if len(keypoints) > 0:
                table_corners = keypoints[0][:, :2]  # first instance, all keypoints, x,y only
                print(f"Frame {frame_num}: Table corners (normalized): {table_corners}")

        frame_num += 1

        # Exit on 'q'
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    return all_results


In [4]:
def average_results(results):
    if len(results) == 0:
        print("No results to average")
        return None

    sum_corners = np.zeros((4, 2), dtype=np.float32)
    count = 0

    for r in results:
        keypoints = r.keypoints.cpu().numpy()
        table_corners = None

        # Case 1: use keypoints if available
        if len(keypoints) > 0:
            candidate = keypoints[0][:, :2]   # take first detection (x, y only)
            if candidate.shape == (4, 2):
                table_corners = candidate

        # Case 2: fallback to bounding box
        if table_corners is None:
            if hasattr(r, "boxes") and len(r.boxes) > 0:
                box = r.boxes[0].xyxy.cpu().numpy()[0]  # (x1, y1, x2, y2)
                x1, y1, x2, y2 = box
                table_corners = np.array([
                    [x1, y1],  # top-left
                    [x2, y1],  # top-right
                    [x2, y2],  # bottom-right
                    [x1, y2],  # bottom-left
                ], dtype=np.float32)

        # Add if we have valid corners
        if table_corners is not None and table_corners.shape == (4, 2):
            sum_corners += table_corners
            count += 1

    if count == 0:
        return None

    avg_corners = sum_corners / count
    return avg_corners


In [5]:
def annotate_frame_with_table(frame, table_corners):
    if table_corners is None:
        return frame  # nothing to draw
    
    # Convert to integer pixel coordinates
    pts = np.int32(table_corners).reshape((-1, 1, 2))

    annotated = frame.copy()
    cv2.polylines(annotated, [pts], isClosed=True, color=(0, 255, 0), thickness=3)

    return annotated


In [15]:
model_path = "TableDetection.pt"
video_path = "../Videos/game_1.mp4"

model = YOLO(model_path)  # load an official model
start_frame = 0
end_frame = 50
results = yolo_on_video(model, video_path, start_frame, end_frame)
avg_corners = average_results(results)
print(f"Average Table corners (normalized): {avg_corners}")
if avg_corners is not None:
    print("Corners being drawn:", avg_corners)
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    ret, frame = cap.read()
    if ret:
        annotated = annotate_frame_with_table(frame, avg_corners)
        cv2.imshow("Average Table Position", annotated)
        cv2.waitKey(0)
    cap.release()
    cv2.destroyAllWindows()
else:
    print("No valid table corners detected in any frame.")


0: 384x640 1 table, 131.1ms
Frame 0: Table corners (normalized): ultralytics.engine.results.Keypoints object with attributes:

conf: array([[    0.50746,     0.58281]], dtype=float32)
data: array([[[     1128.9,      552.46,     0.50746],
        [     809.01,       791.3,     0.58281]]], dtype=float32)
has_visible: True
orig_shape: (1080, 1920)
shape: (1, 2, 3)
xy: array([[[     1128.9,      552.46],
        [     809.01,       791.3]]], dtype=float32)
xyn: array([[[    0.58797,     0.51154],
        [    0.42136,     0.73269]]], dtype=float32)
Speed: 25.9ms preprocess, 131.1ms inference, 12.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 table, 93.7ms
Frame 1: Table corners (normalized): ultralytics.engine.results.Keypoints object with attributes:

conf: array([[    0.50819,     0.58242]], dtype=float32)
data: array([[[     1129.2,       553.1,     0.50819],
        [     805.88,      794.94,     0.58242]]], dtype=float32)
has_visible: True
orig_shape: (1080, 1920)

In [17]:
model_corners = [[353.24, 494.26], [353.24, 679.19], [1431.2, 679.19], [1431.2, 494.26]]
model_corners1 = [[495, 501], [263, 657], [1457, 692], [1305, 524]]
actual_corners = [[509, 525], [303, 644], [1437, 664], [1293, 539]]

model_corners_np = np.array(model_corners1)
actual_corners_np = np.array(actual_corners)
sse = np.sum((model_corners_np - actual_corners_np) ** 2)
print("Sum of squared errors:", sse)

Sum of squared errors: 4094
